# Categorization (Coarse Classification)

From 01-data_cleaning.ipynb we can see that the data contains numerous categorical variables with many different levels. Each level of the category would create a dummy variable in our model, therefore there will be many redundant features in the model due to infrequent categories. We would like to create categories of values such that fewer parameters will have to be estimated and probably a more robust model can be obtained. We are going to categorize the values based on similar odds ratio.

In [1]:
%matplotlib inline

# Filter warnings
import warnings
warnings.filterwarnings("ignore")

# Data manipulation
import pandas as pd
import numpy as np

# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set font scale and style
plt.rcParams.update({'font.size': 15})

# Load data

In [2]:
# Load data
df = pd.read_csv('../data/clean_data.csv')
df.head()

,C_YEAR,C_MNTH,C_WDAY,C_HOUR,FATAL,C_VEHS,C_CONF,C_RCFG,C_WTHR,C_RSUR,...,V_TYPE,V_YEAR,P_ID,P_SEX,P_AGE,P_PSN,P_ISEV,P_SAFE,P_USER,C_CASE
0,1999,January,Monday,9.0,0,2.0,Right turn,At an intersection,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1992.0,1.0,F,33.0,Driver,Injury,Safety device used,Motor Vehicle Driver,2890
1,1999,January,Monday,9.0,0,2.0,Right turn,At an intersection,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1992.0,1.0,F,70.0,Driver,No Injury,Safety device used,Motor Vehicle Driver,2890
2,1999,January,Monday,20.0,0,1.0,Ran off left shoulder,Intersection with parking lot entrance,Clear and sunny,"Dry, normal",...,Light Duty Vehicle,1988.0,1.0,F,38.0,Driver,Injury,Safety device used,Motor Vehicle Driver,4332
3,1999,January,Monday,5.0,0,2.0,Hit a moving object,At an intersection,Raining,Wet,...,Other trucks and vans,1995.0,1.0,M,34.0,Driver,No Injury,Safety device used,Motor Vehicle Driver,5053
4,1999,January,Monday,5.0,0,2.0,Hit a moving object,At an intersection,Raining,Wet,...,Other trucks and vans,1995.0,2.0,M,30.0,"Front row, right outboard",No Injury,Safety device used,Motor Vehicle Passenger,5053


## 1. Collision configuration
We will keep only four categories with the largest odds and aggregate others

In [3]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_CONF', aggfunc = 'count')
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [38.68, 157.85, 106.15, 21.62, 7.53, 38.92, 56.74, 38.08, 98.27, 25.48, 28.71, 281.09, 68.63, 84.74, 44.05, 69.39, 69.81, 121.64]


C_CONF,Any other single-vehicle,Any other two-vehicle - different direction,Any other two-vehicle - same direction,Approaching side-swipe,Head-on collision,Hit a moving object,Hit a parked motor vehicle,Hit a stationary object,Left turn across opposing traffic,Ran off left shoulder,Ran off right shoulder,Rear-end collision,Right angle collision,Right turn,Rollover on roadway,Side swipe,left turn conflict,right turn conflict
FATAL,,,,,,,,,,,,,,,,,,
0,333464,401728,5838,24730,127288,34365,29731,82363,290097,96921,128527,1323634,607543,41355,8458,152307,38883,29072
1,8621,2545,55,1144,16910,883,524,2163,2952,3804,4476,4709,8852,488,192,2195,557,239


In [4]:
coll_dict = {'Hit a moving object':'other', 
             'Hit a stationary object':'other', 
             'Ran off left shoulder':'other',
            'Ran off right shoulder':'other', 
             'Rollover on roadway': 'Rollover on roadway', 
             'Any other single-vehicle ':'other',
            'Right turn':'other',
             'Head-on collision': 'Head-on collision',
             'Rear-end collision': 'Rear-end collision', 
            'left turn conflict':'other',  
             'Left turn across opposing traffic':'other',
            'right turn conflict':'other', 
             'Right angle collision':'other', 
             'Hit a parked motor vehicle':'other',
            'Approaching side-swipe':'other', 
            'Any other two-vehicle - different direction': 'Any other two-vehicle - different direction', 
             'Side swipe':'other', 
            'Any other two-vehicle - same direction':'other'}

df['C_CONF'].replace(coll_dict, inplace = True)

df.C_CONF.value_counts()

other                                          1932149
Rear-end collision                             1328343
Any other two-vehicle - different direction     404273
Head-on collision                               144198
Rollover on roadway                               8650
Name: C_CONF, dtype: int64

## 2. Roadway configuration

In [5]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_RCFG', aggfunc = 'count')
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [107.4, 50.32, 124.5, 119.15, 37.34, 6.91, 41.74, 286.4, 184.57, 33.51]


C_RCFG,At an intersection,"Bridge, overpass, viaduct",Express lane of a freeway system,Intersection with parking lot entrance,Non-intersection,Passing or climbing lane,Railroad level crossing,Ramp,Traffic circle,Tunnel or underpass
FATAL,,,,,,,,,,
0,1996193,34319,249,217095,1483962,318,11561,7160,1292,4155
1,18586,682,2,1822,39738,46,277,25,7,124


In [6]:
roadway_dict = {'Non-intersection': 'other', 
                'At an intersection':'At an intersection', 
                'Intersection with parking lot entrance': 'Intersection with parking lot entrance',
                'Railroad level crossing': 'other',
               'Bridge, overpass, viaduct': 'other', 
                'Tunnel or underpass': 'other', 
                'Passing or climbing lane': 'other',
               'Ramp': 'Express lane of a freeway system', 
                'Traffic circle': 'Express lane of a freeway system',
                'Express lane of a freeway system': 'Express lane of a freeway system'}

df['C_RCFG'].replace(roadway_dict, inplace = True)

df.C_RCFG.value_counts()

At an intersection                        2014779
other                                     1575182
Intersection with parking lot entrance     218917
Express lane of a freeway system             8735
Name: C_RCFG, dtype: int64

## 3. Weather condition

In [7]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_WTHR', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [63.63, 39.32, 50.57, 83.27, 53.22, 31.79, 23.98]


C_WTHR,Clear and sunny,"Freezing rain, sleet, hail","Overcast, cloudy but no precipitation",Raining,Snowing,Strong wind,Visibility limitation
FATAL,,,,,,,
0,2688648,19975,346496,416193,222200,9410,53382
1,42254,508,6852,4998,4175,296,2226


In [8]:
wthr_dict = {'Clear and sunny': 'Clear and sunny',
             'Overcast, cloudy but no precipitation': 'Overcast, cloudy but no precipitation',
              'Raining': 'Raining',
            'Snowing' : 'Snowing', 
             'Freezing rain, sleet, hail' :'other',
              'Visibility limitation':'Visibility limitation',
              'Strong wind': 'other'}

df['C_WTHR'].replace(wthr_dict, inplace = True)

df.C_WTHR.value_counts()

Clear and sunny                          2730902
Raining                                   421191
Overcast, cloudy but no precipitation     353348
Snowing                                   226375
Visibility limitation                      55608
other                                      30189
Name: C_WTHR, dtype: int64

## 4. Road surface

In [9]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_RSUR', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [60.62, 59.33, 49.43, 27.96, 231.67, 21.5, 50.33, 52.1, 76.62]


C_RSUR,"Dry, normal",Flooded,Icy,Muddy,Oil,Sand/gravel/dirt,Slush,Snow,Wet
FATAL,,,,,,,,,
0,2564900,178,204511,2908,695,15694,53899,167814,745705
1,42308,3,4137,104,3,730,1071,3221,9732


In [10]:
roadsur_dict = {'Dry, normal':'Normal', 
                'Wet': 'Wet', 
                'Snow': 'Snow', 
                'Slush':'Snow', 
                'Icy': 'other', 
                'Sand/gravel/dirt':'other',
                'Muddy':'other', 
                 'Oil': 'Oil', 
                'Flooded': 'Normal'}

df['C_RSUR'].replace(roadsur_dict, inplace = True)

df.C_RSUR.value_counts()

Normal    2607389
Wet        755437
other      228084
Snow       226005
Oil           698
Name: C_RSUR, dtype: int64

## 5. Road alignment

In [11]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_RALN', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [28.02, 26.27, 24.51, 78.19, 51.8, 30.45]


C_RALN,Bottom of hill or gradient,Curved and level,Curved with gradient,Straight and level,Straight with gradient,Top of hill or gradient
FATAL,,,,,,
0,16170,235110,138931,2944003,398981,23109
1,577,8949,5668,37653,7703,759


In [12]:
roadall_dist = {'Straight and level':'Straight and level',
                'Straight with gradient':'Straight with gradient', 
                'Curved and level': 'Curved',
               'Curved with gradient':'Curved', 
                'Top of hill or gradient': 'Top of hill or gradient',
                'Bottom of hill or gradient': 'Bottom of hill or gradient'}

df['C_RALN'].replace(roadall_dist, inplace = True)

df.C_RALN.value_counts()

Straight and level            2981656
Straight with gradient         406684
Curved                         388658
Top of hill or gradient         23868
Bottom of hill or gradient      16747
Name: C_RALN, dtype: int64

## 6. Vehicle type

In [13]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'V_TYPE', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [219.67, 41.87, 69.32, 24.29, 44.94, 61.79, 15.41, 12.8, 27.58, 105.57, 116.2, 23.47, 34.91]


V_TYPE,Bicycle,Fire engine,Light Duty Vehicle,Motorcycle and moped,Other trucks and vans,Panel/cargo van,Purpose-built motorhome,Road tractor,School bus,Smaller school bus,Street car,Unit trucks,Urban and Intercity Bus
FATAL,,,,,,,,,,,,,
0,3954,628,3400890,61385,109832,45909,1572,39039,14284,739,1743,44424,31905
1,18,15,49062,2527,2444,743,102,3051,518,7,15,1893,914


In [14]:
vehtype_dict ={'Light Duty Vehicle': 'Light Duty Vehicle', 
               'Other trucks and vans': 'Other trucks and vans', 
               'Urban and Intercity Bus': 'Urban and Intercity Bus',
              'Construction equipment': 'Construction equipment',
               'Bicycle': 'Bicycle', 
               'Unit trucks': 'other', 
               'Road tractor': 'Road tractor',
              'School bus': 'other', 
               'Snowmobile': 'Snowmobile',
               'Motorcycle and moped': 'other',
               'Street car': 'Street car',
              'Panel/cargo van': 'Light Duty Vehicle', 
               'Off road vehicles': 'Off road vehicles', 
               'Farm equipment': 'Farm equipment', 
               'Purpose-built motorhome': 'Road tractor',
              'Smaller school bus': 'Street car',
               'Fire engine':'Other trucks and vans'}

df['V_TYPE'].replace(vehtype_dict, inplace = True)

df.V_TYPE.value_counts()

Light Duty Vehicle         3496604
other                       125031
Other trucks and vans       112919
Road tractor                 43764
Urban and Intercity Bus      32819
Bicycle                       3972
Street car                    2504
Name: V_TYPE, dtype: int64

## 7. Traffic control

In [15]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'C_TRAF', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [33.86, 19.97, 42.12, 24.0, 184.59, 136.73, 20.37, 9.04, 10.14, 226.69, inf, 131.52, 67.31, 214.17, 53.2, 27.97, 154.92]


C_TRAF,Control device not specified,Markings on the road,No control present,No passing zone sign,Pedestrian crosswalk,Police officer,"Railway crossing with signals, or signals and gates",Railway crossing with signs only,Reduced speed zone,School bus stopped with school bus signal lights flashing,School crossing,"School guard, flagman",Stop sign,Traffic signals fully operational,Traffic signals in flashing mode,Warning sign,Yield sign
FATAL,,,,,,,,,,,,,,,,,
0,711,2356,2000882,1512,17167,1504,2159,488,1247,3627,231,17361,467093,1178589,13831,2154,45392
1,21,118,47500,63,93,11,106,54,123,16,0,132,6939,5503,260,77,293


In [16]:
tracon_dict ={'Traffic signals fully operational': 'Traffic signals fully operational',
              'Traffic signals in flashing mode': 'No control present',
             'Stop sign': 'Stop sign',
              'Yield sign': 'Yield sign',
              'Warning sign':'other',
              'Pedestrian crosswalk':'Pedestrian crosswalk',
             'Police officer': 'Police officer', 
              'School guard, flagman': 'All School buses traffic control', 
              'School crossing': 'All School buses traffic control',
              'Reduced speed zone': 'Railway crossing with signs only', 
              'No passing zone sign': 'other', 
              'Markings on the road': 'other',
             'School bus stopped with school bus signal lights flashing': 'All School buses traffic control',
             'Railway crossing with signals, or signals and gates':'other',
             'Railway crossing with signs only': 'Railway crossing with signs only',
             'Control device not specifie': 'Control device not specified',
              'No control present': 'No control present'}

df['C_TRAF'].replace(tracon_dict, inplace = True)

df.C_TRAF.value_counts()

No control present                   2062473
Traffic signals fully operational    1184092
Stop sign                             474032
Yield sign                             45685
All School buses traffic control       21367
Pedestrian crosswalk                   17260
other                                   8545
Railway crossing with signs only        1912
Police officer                          1515
Control device not specified             732
Name: C_TRAF, dtype: int64

## 8. Person position

In [17]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'P_PSN', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [67.12, 47.13, 56.34, 57.5, 15.06, 46.05, 51.66, 58.83, 16.23, 23.02, 26.85, 28.41]


P_PSN,Driver,"Front row, center","Front row, right outboard",Outside passenger compartment,Position unknown,"Second row, center","Second row, left outboard","Second row, right outboard",Sitting on someone’s lap,"Third row, center","Third row, left outboard","Third row, right outboard"
FATAL,,,,,,,,,,,,
0,2546515,50332,675293,345,13807,66306,168619,207805,211,23001,1826,2244
1,37938,1068,11985,6,917,1440,3264,3532,13,999,68,79


In [18]:
perpos_dict = {'Driver': 'Driver', 
               'Front row, right outboard': 'Front row',
               'Pedestrian': 'Pedestrian',
               'Second row, right outboard': 'Second row',
              'Second row, right outboard': 'Outside passenger compartment', 
               'Second row, left outboard': 'Second row',
              'Second row, center': 'Second row', 
               'Front row, center': 'Front row',
               'Position unknown': 'Position unknown',
              'Third row, center': 'Third row', 
               'Third row, left outboard': 'Third row',
              'Third row, right outboard': 'Third row', 
               'Sitting on someone’s lap': 'Position unknown'}

df['P_PSN'].replace(perpos_dict, inplace = True)

df.P_PSN.value_counts()

Driver                           2584453
Front row                         738678
Second row                        239629
Outside passenger compartment     211688
Third row                          28217
Position unknown                   14948
Name: P_PSN, dtype: int64

## 9. Safety device used

In [19]:
pvt = df.pivot_table('C_CASE', index = 'FATAL', columns = 'P_SAFE', aggfunc = 'count', fill_value = 0)
print("Odd ratio:", list(np.around(pvt.values[0]/pvt.values[1],2)))
pvt

Odd ratio: [23.93, 30.31, 8.34, 34.33, 21.0, 77.7]


P_SAFE,Helmet worn,No safety device equipped,No safety device used,Other safety device used,Reflective clothing worn,Safety device used
FATAL,,,,,,
0,55904,49429,93219,23241,21,3534490
1,2336,1631,11174,677,1,45490


In [20]:
safe_dict = {'Safety device used': 'Safety device used', 
             'No safety device used': 'No safety device used', 
             'No safety device equipped': 'No safety device equipped',
            'Other safety device used': 'Other safety device used',
             'Helmet worn': 'Helmet worn', 
            'Reflective clothing worn' : 'Helmet worn',
            'Both helmet and reflective clothing used': 'Helmet worn'}

df['P_SAFE'].replace(safe_dict, inplace = True)

df.P_SAFE.value_counts()

Safety device used           3579980
No safety device used         104393
Helmet worn                    58262
No safety device equipped      51060
Other safety device used       23918
Name: P_SAFE, dtype: int64

# Save data as CSV

In [21]:
df.to_csv('../data/ml_data.csv', index = False)